Последний шаблон, который мы разберём — "посетитель" (visitor).
В домашнем задании потребуется его реализовать в ЯТЬ.

Рассмотрим какую-нибудь иерархию домашних животных с четырьмя операциями. Животное можно погладить по голове, по животу, побаловать (pet) и узнать, безопасно ли находиться рядом. Для котов и собак эти операции приводят к разным эффектам. Более того, у кота можно узнать уровень счастья, а у собаки — посмотреть на хвост.

In [5]:
import random
from abc import ABCMeta, abstractmethod

class Pet(metaclass=ABCMeta):
    @abstractmethod
    def pat_head(self):
        pass

    @abstractmethod
    def rub_belly(self):
        pass

    @abstractmethod
    def pet(self):
        pass

    @abstractmethod
    def is_safe(self):
        pass


class Cat(Pet):
    def pat_head(self):
        print("Purr!")

    def rub_belly(self):
        print("Don't you dare!")

    def happiness(self):
        return random.randint(1, 5)

    def pet(self):
        self.pat_head()

    def is_safe(self):
        return self.happiness() >= 3

class Dog(Pet):
    def pat_head(self):
        print("I'm happy!")

    def rub_belly(self):
        print("I'm very happy!")

    def tail_wagging(self):
        return True
    
    def offer_bone(self):
        pass

    def pet(self):
        self.rub_belly()

    def is_safe(self):
        return self.tail_wagging()

In [7]:
def test_pet(pet):
    pet.pet();
    print(pet.is_safe())
    print(pet.is_safe())
    print(pet.is_safe())

test_pet(Cat())
test_pet(Dog())

Purr!
True
True
True
I'm very happy!
True
True
True


Если мы добавляем новый тип животного, то всё просто: унаследовались от `Pet`, определили методы. А вот если мы захотим добавить новую операцию (например, "попробовать сделать так, чтобы рядом стало безопасно"), то нам придётся либо менять все классы и всю иерархию, либо разбирать случаи:

In [9]:
def try_make_safe(pet):
    if isinstance(pet, Cat):
        print('Impossible')
    elif isinstance(pet, Dog):
        if not pet.is_safe():
            pet.offer_bone()
        assert pet.is_safe()
    else:
        raise NotImplementedError

In [10]:
try_make_safe(Cat())

Impossible


In [11]:
try_make_safe(Dog())

Так мы можем легко добавлять новые методы, но в каждом из них у нас идёт длинная цепочка `if`'ов, в которой легко забыть случай (и компилятор это никак не проверит!). Более того, нам требуется динамически узнавать тип объекта, что может быть сложно в некоторых языках вроде C++.

Чтобы можно было и добавлять новые методы без изменения классов, и получать помощь от компилятора, есть паттерн "посетитель". Мы заводим абстракцию "операция над животным" и реализуем нашу функцию в её терминах:

In [18]:
class PetVisitor(metaclass=ABCMeta):
    @abstractmethod
    def visit_cat(self, cat):
        pass

    @abstractmethod
    def visit_dog(self, dog):
        pass
    
class TryMakeSafeVisitor(PetVisitor):
    def visit_cat(self, cat):
        print('Impossible')
        
    def visit_dog(self, dog):
        if not dog.is_safe():
            dog.offer_bone()
        assert dog.is_safe()

А теперь мы один раз добавляем в исходные классы новый метод `accept`, который вызовет нужный метод из visitor'а:

In [15]:
# По-хорошему надо скопировать определение классов, но это сильно раздует код
Cat.accept = lambda self, visitor: visitor.visit_cat(self)
Dog.accept = lambda self, visitor: visitor.visit_dog(self)

In [16]:
Cat().accept(TryMakeSafeVisitor())

Impossible


In [20]:
Dog().accept(TryMakeSafeVisitor())

У нас появился способ добавлять новые "методы" ко всем реализациям интерфейса, не меняя исходные классы. Точнее, мы один раз добавили им метод `accept`, а теперь кто угодно может добавлять новое поведение.

В качестве бонуса мы получаем статическую проверку типов: в языках вроде C++/Java в функции `try_make_safe` потребовалось бы после проверки на тип явно писать приведение типов, чтобы вызвать метод `offer_bone()`. А в случае с классом у нас есть отдельные методы, и у каждого метода параметр имеет понятный тип: либо `Cat`, либо `Dog`. И компилятор может понять, что происходит.

Кстати, так как в Python нет перегрузки функций, то мне пришлось писать длинные имена: `visit_cat`, `visit_dog`... В Java и C++ обычно будет один метод `visit()` с параметром разных типов. Тогда компилятор заодно проверит, что нет пересечений.

Есть две тонкости:

1. Если мы захотим добавить новый класс, придётся изменять все visitor'ы. То есть добавлять "методы" теперь стало просто, а вот новые классы — сложно.
1. Непонятно, как возвращать значение из `visit()`, особенно в языках со статической типизаций. Там надо один раз описать `PetVisitor` и указать возвращаемый тип. Обычно ставят просто `void` и говорят, что `visit`/`accept` ничего не возвращают, а если уж и надо — то visitor запоминает значение внутри себя. В случае с Python можно сказать, что `accept()` просто возвращает то, что вернул `visit()`, а `visit()` возвращает, что захочет. Это, конечно, может несколько свести с ума среду разработки: ей придётся догадываться, что в данном конкретном случае `accept` вызовет конкретный visitor, все методы которого всегда возвращают объект определённого типа.

В домашнем задании вам потребуется добавить в интерпретатор ЯТЬ интерфейс `NodeVisitor`, метод `accept` для всех узлов, а также реализовать два visitor'а.